# Reflection

In [1]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


_set_if_undefined("TAVILY_API_KEY")


## Generate
a 5 paragraph essay

In [5]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOllama(
    model="llama3.1", max_tokens=32768
)
generate = prompt | llm

In [6]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

The Little Prince, written by Antoine de Saint-Exupéry, has been a beloved children's tale for generations. Despite being penned over 70 years ago, this novella remains remarkably relevant in modern childhood. One reason for its enduring relevance lies in its timeless exploration of the human condition.

Through the Little Prince's journey, de Saint-Exupéry masterfully captures the complexities and simplicity of childhood. The prince's relationships with his own planet, the rose, and various grown-ups on Earth serve as poignant reminders that childhood is a fleeting period marked by innocence, curiosity, and a deep-seated longing for connection. These universal themes remain pertinent today, as children continue to grapple with issues such as identity, belonging, and the meaning of life.

Furthermore, The Little Prince's depiction of the adult world serves as a powerful commentary on the societal pressures placed upon modern childhood. De Saint-Exupéry skewers the artificial constructs

## Reflect

In [7]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [8]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

**Overall Impression**

Your essay provides a thoughtful and engaging analysis of the relevance of The Little Prince in modern childhood. You've done a great job of exploring the novella's themes and messages, and demonstrating their continued significance for young readers.

**Strengths**

* Your writing is clear and concise, making it easy to follow your argument.
* You've effectively used specific examples from the text to support your claims.
* Your analysis of The Little Prince's depiction of adult society is particularly insightful, highlighting the novella's commentary on societal pressures and consumer culture.
* You've also done a great job of emphasizing the universality of childhood experiences and the importance of reevaluating priorities.

**Weaknesses**

* While your essay is well-written, it feels a bit too concise. Consider expanding to 6-8 paragraphs to provide more depth and detail in each section.
* Some of your sentences feel a bit generic or cliche ("The Little Pri

## Repeat

In [9]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Thank you for the detailed critique! I'm glad to see that my essay was well-received overall. I'll take your feedback into consideration and revise my essay to address the weaknesses you mentioned.

Here's a revised version of the essay:

The Little Prince, written by Antoine de Saint-Exupéry, has been a beloved children's tale for generations. Despite being penned over 70 years ago, this novella remains remarkably relevant in modern childhood. One reason for its enduring relevance lies in its timeless exploration of the human condition.

As we delve into the story, we find that the Little Prince's relationship with his own planet and rose is a poignant reminder of the complexities of childhood. On his own small planet, the prince lived in harmony with the rose, which represented love and companionship. This idyllic existence was disrupted when the king arrived, and the prince realized that his world was not as perfect as he had thought. The experience taught him to appreciate the beau

## Define graph

In [10]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [11]:
config = {"configurable": {"thread_id": "1"}}

In [12]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the topicality of The Little Prince and its message in modern life"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content="Here is a five-paragraph essay on the topicality of The Little Prince and its message in modern life:\n\nThe Little Prince, written by Antoine de Saint-Exupéry in 1943, has long been considered a timeless tale for children. However, beneath its whimsical surface lies a profound and enduring exploration of human nature that remains remarkably relevant today. As the world grapples with complex issues such as climate change, social media addiction, and existential crises, The Little Prince's message of love, connection, and introspection resonates powerfully with readers of all ages.\n\nOne of the most striking aspects of The Little Prince is its portrayal of the shallow, materialistic society that exists on Earth. Through the character of the Rose, who is loved by the Little Prince but ultimately destroyed by his own vanity, de Saint-Exupéry critiques the ways in which humans tend to prioritize superficial relationships and possessions over g

In [13]:
state = graph.get_state(config)

In [14]:
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

Generate an essay on the topicality of The Little Prince and its message in modern life

================================== Ai Message ==================================

Here is a five-paragraph essay on the topicality of The Little Prince and its message in modern life:

The Little Prince, written by Antoine de Saint-Exupéry in 1943, has long been considered a timeless tale for children. However, beneath its whimsical surface lies a profound and enduring exploration of human nature that remains remarkably relevant today. As the world grapples with complex issues such as climate change, social media addiction, and existential crises, The Little Prince's message of love, connection, and introspection resonates powerfully with readers of all ages.

One of the most striking aspects of The Little Prince is its portrayal of the shallow, materialistic society that exists on Earth. Through the character of the 